In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn.apionly as sns
import datetime
import warnings
%matplotlib inline
plt.style.use('ggplot')
warnings.filterwarnings('ignore')

In [18]:
def get_clean_data(df_original):
    df = df_original.copy(deep=True)
    df['date'] =  pd.to_datetime(df['datetime'].apply(lambda x: x[:10]), format='%Y-%m-%d')
    df['year'] = df['date'].dt.year
    df['time'] = pd.to_datetime(df['datetime'].apply(lambda x: x[:19])).dt.time
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hour'] = df['datetime'].apply(lambda x: x[11:13]).astype(int)
    df['minute'] = df['datetime'].apply(lambda x: x[14:16]).astype(int)
    df['weekday'] = df['date'].dt.dayofweek
    df['season'] = np.where(df['month'].isin(list(range(4,10))), 'V', 'I')
    df['date_hour'] = df.apply(lambda x: datetime.datetime.combine(x['date'], x['time']), axis=1)
    df.set_index('date_hour', inplace=True)
    df = df[df.index < '2017']
    clean_df = df[['date', 'year', 'month', 'season', 'day','weekday','time', 'hour', 'minute', 'value']]
    
    return clean_df

### df with secondary band and spot price 

In [26]:
raw_band_price = pd.read_csv('export_PrecioBandaDeRegulaciónSecundaria_2017-01-15_19-11.csv', encoding='latin1', delimiter=';')
band_price = get_clean_data(raw_band_price)
band_price = band_price.rename(columns={'value':'band'})

In [27]:
raw_spot_price = pd.read_csv('export_PrecioMercadoSPOTDiario_2017-02-02_09-47.csv', encoding='latin1', delimiter=';')
raw_spot_price = raw_spot_price[raw_spot_price['geoid'] == 3]
spot_price = get_clean_data(raw_spot_price)
spot_price = spot_price.rename(columns={'value':'spot'})

In [28]:
#Merge both df
spot_band = band_price.merge(spot_price[['spot']], how='left', left_index=True, right_index=True)

In [30]:
#Clean df in order to train a model, output: band price
spot_band = spot_band[['hour', 'weekday', 'season', 'spot', 'band']]

In [23]:
#Create dummy variables
for feature in ['hour', 'weekday', 'season']:
    spot_band[pd.get_dummies(spot_band[feature], drop_first=False).columns.tolist()] = pd.get_dummies(spot_band[feature], drop_first=False)

In [25]:
spot_band.columns

Index([   'hour', 'weekday',  'season',    'spot',    'band',         0,
               1,         2,         3,         4,         5,         6,
               7,         8,         9,        10,        11,        12,
              13,        14,        15,        16,        17,        18,
              19,        20,        21,        22,        23,       'I',
             'V'],
      dtype='object')

In [32]:
weekday_dict = {
        '0':'L', '1':'M', 2:'X', 3:'J', 4:'V', 5:'S', 6:'D' 
}

In [37]:
spot_band.replace({'weekday_dict':weekday_dict})

,hour,weekday,season,spot,band
date_hour,,,,,
2014-01-01 00:00:00,0,2,I,20.02,69.01
2014-01-01 01:00:00,1,2,I,10.34,50.00
2014-01-01 02:00:00,2,2,I,5.35,57.80
2014-01-01 03:00:00,3,2,I,5.00,54.15
2014-01-01 04:00:00,4,2,I,0.50,54.15
2014-01-01 05:00:00,5,2,I,0.00,54.82
2014-01-01 06:00:00,6,2,I,0.00,55.00
2014-01-01 07:00:00,7,2,I,0.00,55.00
2014-01-01 08:00:00,8,2,I,0.00,55.00
